In [7]:
import numpy as np
import pandas as pd
import scipy as sc
import os
from datetime import date
from os import listdir
from os.path import isfile, join
import math
from scipy.optimize import curve_fit

In [8]:
def wistar_w(g, nu):
    if g == str("male"):
        x = nu-17
        y = [491.7, 438.3, 486.1, 572, 475.2, 468.6, 446, 431.7, 452.9, 413,
      417.7, 491.1, 495.9, 500.3, 548.2, 524.7]

    elif g == str("female"):
        x = nu-1
        y = [282.7, 340.6, 231.3, 301.9, 325.6, 328.7, 297.8, 295.2,
      306.7, 287.2, 276, 293.6, 301.9, 281.7]
    else:
        print("You fucked up the gender on weight optimizer.")
    return y[x]

In [78]:
def d_day_index(f, n):
    for i1 in range(n):
        f.loc[i1, ['day_index']] = i1+1
    return f


def state_hour(f, n):
    for i2 in range(n + 1):
        dn = 24 * i2
        f.loc[dn:dn + 6, ['state']] = str('dark')
        f.loc[dn + 7:dn + 18, ['state']] = str('light')
        f.loc[dn + 19:dn + 24, ['state']] = str('dark')
    return f


def m_day_and_hour_index(f, n):
    h_counter = 1
    for i3 in range(n + 1):
        dn = 24 * i3
        f.loc[dn:dn + 23, ['day_index']] = i3+1
        for j3 in range(24):
            interval_m = dn + j3
            f.loc[interval_m:interval_m + 1, ['hour_index']] = h_counter
            h_counter = h_counter + 1
    return f


def state_(f, n):
    for i4 in range(n + 1):
        dn = 1440 * i4
        f.loc[dn:dn + 419, ['state']] = str('dark')
        f.loc[dn + 420:dn + 1139, ['state']] = str('light')
        f.loc[dn + 1140:dn + 1439, ['state']] = str('dark')
    return f


def day_and_hour_index(f, n):
    h_counter = 1
    for i5 in range(n + 1):
        dn = 1440 * i5
        f.loc[dn:dn + 1439, ['day_index']] = i5+1
        for j5 in range(24):
            interval_m = dn + (j5*60)
            f.loc[interval_m:interval_m + 60, ['hour_index']] = h_counter
            h_counter = h_counter + 1
    return f

def threshup_():
    #DOI 10.1211/jpp.60.1.0008
    th_u_comft = 3.38 + 0.52
    #th_u_stretched = 4.63 + 0.44
    #th_u_burst = 6.63 + 0.92
    return th_u_comft


def real_consump(val, w, p):
    a = np.nan
    if (p == 5):
        a = (val*0.05*0.789)/(w) *1000
    elif (p == 10):
        a = (val*0.1*0.789)/(w) *1000
    elif ( p == 20 ):
        a = (val*0.2*0.789)/(w) *1000
    else:
        print("You messed up the input in real_consump function")
    return(a)



def writer_(main_df, list_of_them, number, method, a_n, path):
    #print(number)
    file = pd.read_excel("{}\{}".format(path, list_of_them[number]), skiprows=35)
    #print(file)
    if method == str('dep_onlywater'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i6 in range(len(extracted_a1)):
            ind0 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i6]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i6]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind0, 'water'] = extracted_a1['water'][list_index[i6]]
    elif method == str('ade_only'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5',
                                     '[ml].2': 'alcohol_10', '[ml].3': 'alcohol_20'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i7 in range(len(extracted_a1)):
            ind1 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i7]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i7]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind1, 'water'] = extracted_a1['water'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_5'] = extracted_a1['alcohol_5'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_10'] = extracted_a1['alcohol_10'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_20'] = extracted_a1['alcohol_20'][list_index[i7]]
    elif method == str('ade_oxy'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5',
                                     '[ml].2': 'alcohol_10', '[ml].3': 'alcohol_20'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i8 in range(len(extracted_a1)):
            ind2 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i8]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i8]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind2, 'water'] = extracted_a1['water'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_5'] = extracted_a1['alcohol_5'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_10'] = extracted_a1['alcohol_10'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_20'] = extracted_a1['alcohol_20'][list_index[i8]]
            main_df.loc[ind2, 'oxytocin'] = str('applied')
    elif method == str('dep_quinine'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i9 in range(len(extracted_a1)):
            ind3 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i9]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i9]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind3, 'water'] = extracted_a1['water'][list_index[i9]]
            main_df.loc[ind3, 'quinine'] = str('applied')
    elif method == str('ade_quinine'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5',
                                     '[ml].2': 'alcohol_10', '[ml].3': 'alcohol_20'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i10 in range(len(extracted_a1)):
            ind4 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i10]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i10]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind4, 'water'] = extracted_a1['water'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_5'] = extracted_a1['alcohol_5'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_10'] = extracted_a1['alcohol_10'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_20'] = extracted_a1['alcohol_20'][list_index[i10]]
            main_df.loc[ind4, 'quinine'] = str('applied')
    else:
        print("{} is non of them".format(list_of_them[number]))
    return main_df

In [87]:
animal = 1030
box = 30
gender = str("male")
strain = str("wistar")

In [88]:
date_i = date(2021, 9, 1)
date_f = date(2021, 9, 7)

delta = date_f - date_i
number_of_days = int(delta.days)+1
print("Experiement was {} days long".format(number_of_days))

total_length = number_of_days * 1440      #length of dataframe for minute time_scale
df = pd.DataFrame(index = range(total_length), columns = ['date', 'time', 'day_index', 'hour_index',
                                                         'animal', 'box', 'strain',
                                                         'state', 'oxytocin', 'quinine', 'water',
                                                         'alcohol_5', 'alcohol_10', 'alcohol_20',
                                                         'locomotive'])

f = state_(df, number_of_days)
df = day_and_hour_index(df, number_of_days)
df['animal'] = animal
df['box'] = box
df['strain'] = strain
df['date'] = pd.date_range("2021-09-01", periods=total_length, freq="T").strftime('%Y-%m-%d')
df['time'] = pd.date_range("2021-09-01", periods=total_length, freq="T").strftime('%H:%M:%S')

path_ = os.getcwd()
path_ = path_+'\\data\\2021'
onlyfiles = [f for f in listdir("{}".format(path_))
             if isfile(join("{}".format(path_), f))]

Experiement was 7 days long


In [89]:
df = writer_(df, onlyfiles, 29, str('ade_only'), box, path_)

df1 = pd.DataFrame(index=range(total_length), columns=['date', 'time', 'day_index', 'hour_index', 'animal', 'box',
                                                       'strain', 'state', 'oxytocin', 'quinine', 'water',
                                                       'alcohol_5', 'alcohol_10', 'alcohol_20', 'locomotive'])


In [90]:
df1 = state_(df1, number_of_days)
df1 = day_and_hour_index(df1, number_of_days)
df1['animal'] = animal
df1['box'] = box
df1['strain'] = strain
df1['date'] = pd.date_range("2021-09-01", periods=total_length, freq="T").strftime('%Y-%m-%d')
df1['time'] = pd.date_range("2021-09-01", periods=total_length, freq="T").strftime('%H:%M:%S')
df1['quinine'] = df['quinine']
df1['oxytocin'] = df['oxytocin']
df1['state'] = df['state']

for hour in range(1, number_of_days * 24 + 1):
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['water'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'water'] - df.loc[list_[i], 'water']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'water'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_5'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_5'] - df.loc[list_[i], 'alcohol_5']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_5'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_10'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_10'] - df.loc[list_[i], 'alcohol_10']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_10'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_20'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_20'] - df.loc[list_[i], 'alcohol_20']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_20'] = temp_val

In [91]:
threshold_low = 0.08
threshold_up = threshup_()
total_length_h = number_of_days * 24

df_h = pd.DataFrame(index=range(number_of_days * 24), columns=['date', 'time', 'day_index', 'hour_index', 'animal',
                                                               'box', 'strain', 'state', 'oxytocin', 'quinine',
                                                               'water', 'alcohol_5', 'alcohol_10', 'alcohol_20', 'locomotive'])
df_h = m_day_and_hour_index(df_h, number_of_days)
df_h = state_hour(df_h, number_of_days)

df_h['animal'] = animal
df_h['box'] = box
df_h['strain'] = strain
df_h['date'] = pd.date_range("2021-09-01", periods=number_of_days * 24, freq="H").strftime('%Y-%m-%d')
df_h['time'] = pd.date_range("2021-09-01", periods=number_of_days * 24, freq="H").strftime('%H:%M:%S')

hours = sorted(set(np.array(df1.loc[(df1['quinine'] == str('applied'))].hour_index)))
for i in range(len(hours)):
    df_h.loc[hours[i]-1, 'quinine'] = str('applied')

hours = sorted(set(np.array(df1.loc[(df1['oxytocin'] == str('applied'))].hour_index)))
for i in range(len(hours)):
    df_h.loc[hours[i]-1, 'oxytocin'] = str('applied')


for hour in range(1, number_of_days*24+1):
    w_w = wistar_w(gender, box)
    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            df_h.loc[ind, 'water'] = _temp
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_5'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up ):
            temp__ = real_consump(_temp, w_w, 5)
            df_h.loc[ind, 'alcohol_5'] = temp__
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_10'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            temp__ = real_consump(_temp, w_w, 10)
            df_h.loc[ind, 'alcohol_10'] = temp__
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_20'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            temp__ = real_consump(_temp, w_w, 20)
            df_h.loc[ind, 'alcohol_20'] = temp__
            _temp = np.nan


In [95]:
df_d_light = pd.DataFrame(index=range(number_of_days), columns=['date', 'day_index', 'animal', 'box',
                                                                'strain', 'state', 'oxytocin', 'quinine', 'water',
                                                                'alcohol_5', 'alcohol_10', 'alcohol_20', 'locomotive'])

df_d_dark = pd.DataFrame(index=range(number_of_days), columns=['date', 'day_index', 'animal', 'box',
                                                               'strain', 'state', 'oxytocin', 'quinine',
                                                               'water', 'alcohol_5', 'alcohol_10', 'alcohol_20', 'locomotive'])

df_d_light = d_day_index(df_d_light, number_of_days)
df_d_dark = d_day_index(df_d_dark, number_of_days)

df_d_light['animal'] = animal
df_d_light['box'] = box
df_d_light['strain'] = strain
df_d_light['state'] = str('light')
df_d_light['date'] = pd.date_range("2021-09-01", periods=number_of_days, freq="D").strftime('%Y-%m-%d')

df_d_dark['animal'] = animal
df_d_dark['box'] = box
df_d_dark['strain'] = strain
df_d_dark['state'] = str('dark')
df_d_dark['date'] = pd.date_range("2021-09-01", periods=number_of_days, freq="D").strftime('%Y-%m-%d')


days = sorted(set(np.array(df1.loc[(df1['quinine'] == str('applied'))].day_index)))
for i in range(len(days)):
    df_d_dark.loc[days[i]-1, 'quinine'] = str('applied')
    df_d_light.loc[days[i]-1, 'quinine'] = str('applied')

days = sorted(set(np.array(df1.loc[(df1['oxytocin'] == str('applied'))].day_index)))
for i in range(len(days)):
    df_d_dark.loc[days[i]-1, 'oxytocin'] = str('applied')
    df_d_light.loc[days[i]-1, 'oxytocin'] = str('applied')


for day in range(1, number_of_days+1):
    list_ = []
    ind = np.array(df_d_light.loc[(df_d_light['day_index'] == day)].index)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'water'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'water'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_5'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_5'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_5'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_5'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_10'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_10'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_10'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_10'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_20'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_20'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_20'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_20'] = np.nansum(temp_arr)



In [ ]:
df1.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_{}_consumptiontt.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_h.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_{}_hourly.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_d_dark.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_daily_dark.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_d_light.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_daily_light.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)

In [101]:
onlyfiles

[]

In [105]:
path_

'C:\\Users\\mehrd\\PycharmProjects\\drinkometer\\data\\2021\\dataframes'

In [409]:
path_ = 'C:\\Users\\mehrd\\PycharmProjects\\drinkometer\\data\\2021\\dataframes\\26'
onlyfiles = [f for f in listdir("{}".format(path_))
             if isfile(join("{}".format(path_), f))]

In [424]:
path_ = 'C:\\Users\\mehrd\\PycharmProjects\\drinkometer\\data\\2021\\dataframes\\25'
#df_m = pd.read_excel("{}\drinkometer_df_25.xlsx".format(path_))

In [425]:
gender = ['female']*14 + ['male']*16
animals = np.concatenate([np.arange(1001,1007), np.arange(1008,1016), np.arange(1017,1033)])
state = [str('dark'), str('light')]


In [179]:
df_d = np.array([])

In [107]:
df1_d = pd.read_excel("{}\drinkometer_df_wistar_{}_{}_daily_{}.xlsx".format(path_, gender, animal, state))

In [177]:
df_d

array([], dtype=float64)

In [426]:
df1_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1001_daily_{}.xlsx".format(path_, gender[0], state[0]))
df2_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1002_daily_{}.xlsx".format(path_, gender[1],state[0]))
df3_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1003_daily_{}.xlsx".format(path_, gender[2],state[0]))
df4_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1004_daily_{}.xlsx".format(path_, gender[3],state[0]))
df5_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1005_daily_{}.xlsx".format(path_, gender[4],state[0]))
df6_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1006_daily_{}.xlsx".format(path_, gender[5],state[0]))
df8_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1008_daily_{}.xlsx".format(path_, gender[6],state[0]))
df9_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1009_daily_{}.xlsx".format(path_, gender[7],state[0]))
df10_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1010_daily_{}.xlsx".format(path_, gender[8],state[0]))
df11_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1011_daily_{}.xlsx".format(path_, gender[9],state[0]))
df12_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1012_daily_{}.xlsx".format(path_, gender[10],state[0]))
df13_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1013_daily_{}.xlsx".format(path_, gender[11],state[0]))
df14_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1014_daily_{}.xlsx".format(path_, gender[12],state[0]))
df15_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1015_daily_{}.xlsx".format(path_, gender[13],state[0]))
df17_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1017_daily_{}.xlsx".format(path_, gender[14],state[0]))
df18_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1018_daily_{}.xlsx".format(path_, gender[15],state[0]))
df19_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1019_daily_{}.xlsx".format(path_, gender[16],state[0]))
df20_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1020_daily_{}.xlsx".format(path_, gender[17],state[0]))
df21_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1021_daily_{}.xlsx".format(path_, gender[18],state[0]))
df22_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1022_daily_{}.xlsx".format(path_, gender[19],state[0]))
df23_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1023_daily_{}.xlsx".format(path_, gender[20],state[0]))
df24_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1024_daily_{}.xlsx".format(path_, gender[21],state[0]))
df25_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1025_daily_{}.xlsx".format(path_, gender[22],state[0]))
df26_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1026_daily_{}.xlsx".format(path_, gender[23],state[0]))
df27_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1027_daily_{}.xlsx".format(path_, gender[24],state[0]))
df28_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1028_daily_{}.xlsx".format(path_, gender[25],state[0]))
df29_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1029_daily_{}.xlsx".format(path_, gender[26],state[0]))
df30_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1030_daily_{}.xlsx".format(path_, gender[27],state[0]))
df31_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1031_daily_{}.xlsx".format(path_, gender[28],state[0]))
df32_d = pd.read_excel("{}\drinkometer_df_wistar_{}_1032_daily_{}.xlsx".format(path_, gender[29],state[0]))


df1_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1001_daily_{}.xlsx".format(path_, gender[0], state[1]))
df2_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1002_daily_{}.xlsx".format(path_, gender[1],state[1]))
df3_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1003_daily_{}.xlsx".format(path_, gender[2],state[1]))
df4_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1004_daily_{}.xlsx".format(path_, gender[3],state[1]))
df5_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1005_daily_{}.xlsx".format(path_, gender[4],state[1]))
df6_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1006_daily_{}.xlsx".format(path_, gender[5],state[1]))
df8_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1008_daily_{}.xlsx".format(path_, gender[6],state[1]))
df9_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1009_daily_{}.xlsx".format(path_, gender[7],state[1]))
df10_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1010_daily_{}.xlsx".format(path_, gender[8],state[1]))
df11_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1011_daily_{}.xlsx".format(path_, gender[9],state[1]))
df12_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1012_daily_{}.xlsx".format(path_, gender[10],state[1]))
df13_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1013_daily_{}.xlsx".format(path_, gender[11],state[1]))
df14_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1014_daily_{}.xlsx".format(path_, gender[12],state[1]))
df15_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1015_daily_{}.xlsx".format(path_, gender[13],state[1]))
df17_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1017_daily_{}.xlsx".format(path_, gender[14],state[1]))
df18_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1018_daily_{}.xlsx".format(path_, gender[15],state[1]))
df19_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1019_daily_{}.xlsx".format(path_, gender[16],state[1]))
df20_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1020_daily_{}.xlsx".format(path_, gender[17],state[1]))
df21_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1021_daily_{}.xlsx".format(path_, gender[18],state[1]))
df22_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1022_daily_{}.xlsx".format(path_, gender[19],state[1]))
df23_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1023_daily_{}.xlsx".format(path_, gender[20],state[1]))
df24_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1024_daily_{}.xlsx".format(path_, gender[21],state[1]))
df25_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1025_daily_{}.xlsx".format(path_, gender[22],state[1]))
df26_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1026_daily_{}.xlsx".format(path_, gender[23],state[1]))
df27_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1027_daily_{}.xlsx".format(path_, gender[24],state[1]))
df28_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1028_daily_{}.xlsx".format(path_, gender[25],state[1]))
df29_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1029_daily_{}.xlsx".format(path_, gender[26],state[1]))
df30_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1030_daily_{}.xlsx".format(path_, gender[27],state[1]))
df31_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1031_daily_{}.xlsx".format(path_, gender[28],state[1]))
df32_l = pd.read_excel("{}\drinkometer_df_wistar_{}_1032_daily_{}.xlsx".format(path_, gender[29],state[1]))


In [427]:
df_m = pd.DataFrame(index=range(2*7), columns=['date', 'day_index', 'strain', 'state', 'oxytocin', 'quinine',
                                                'w_1', 'a5_1', 'a10_1', 'a20_1', 'loco_1',
                                                'w_2', 'a5_2', 'a10_2', 'a20_2', 'loco_2',
                                                'w_3', 'a5_3', 'a10_3', 'a20_3', 'loco_3',
                                                'w_4', 'a5_4', 'a10_4', 'a20_4', 'loco_4',
                                                'w_5', 'a5_5', 'a10_5', 'a20_5', 'loco_5',
                                                'w_6', 'a5_6', 'a10_6', 'a20_6', 'loco_6',
                                                'w_8', 'a5_8', 'a10_8', 'a20_8', 'loco_8',
                                                'w_9', 'a5_9', 'a10_9', 'a20_9', 'loco_9',
                                                'w_10', 'a5_10', 'a10_10', 'a20_10', 'loco_10',
                                                'w_11', 'a5_11', 'a10_11', 'a20_11', 'loco_11',
                                                'w_12', 'a5_12', 'a10_12', 'a20_12', 'loco_12',
                                                'w_13', 'a5_13', 'a10_13', 'a20_13', 'loco_13',
                                                'w_14', 'a5_14', 'a10_14', 'a20_14', 'loco_14',
                                                'w_15', 'a5_15', 'a10_15', 'a20_15', 'loco_15',
                                                'w_17', 'a5_17', 'a10_17', 'a20_17', 'loco_17',
                                                'w_18', 'a5_18', 'a10_18', 'a20_18', 'loco_18',
                                                'w_19', 'a5_19', 'a10_19', 'a20_19', 'loco_19',
                                                'w_20', 'a5_20', 'a10_20', 'a20_20', 'loco_20',
                                                'w_21', 'a5_21', 'a10_21', 'a20_21', 'loco_21',
                                                'w_22', 'a5_22', 'a10_22', 'a20_22', 'loco_22',
                                                'w_23', 'a5_23', 'a10_23', 'a20_23', 'loco_23',
                                                'w_24', 'a5_24', 'a10_24', 'a20_24', 'loco_24',
                                                'w_25', 'a5_25', 'a10_25', 'a20_25', 'loco_25',
                                                'w_26', 'a5_26', 'a10_26', 'a20_26', 'loco_26',
                                                'w_27', 'a5_27', 'a10_27', 'a20_27', 'loco_27',
                                                'w_28', 'a5_28', 'a10_28', 'a20_28', 'loco_28',
                                                'w_29', 'a5_29', 'a10_29', 'a20_29', 'loco_29',
                                                'w_30', 'a5_30', 'a10_30', 'a20_30', 'loco_30',
                                                'w_31', 'a5_31', 'a10_31', 'a20_31', 'loco_31',
                                                'w_32', 'a5_32', 'a10_32', 'a20_32', 'loco_32'])


In [428]:
for i in range(7):
    df_m.loc[2*i, 'state'] = str('dark')
    df_m.loc[2*i+1, 'state'] = str('light')
    df_m.loc[2*i, 'date'] = df1_l.loc[i, 'date']
    df_m.loc[2*i+1, 'date'] = df1_l.loc[i, 'date']
    df_m.loc[2*i, 'day_index'] = df1_l.loc[i, 'day_index']
    df_m.loc[2*i+1, 'day_index'] = df1_l.loc[i, 'day_index']
    df_m.loc[2*i, 'strain'] = df1_l.loc[i, 'strain']
    df_m.loc[2*i+1, 'strain'] = df1_l.loc[i, 'strain']
    df_m.loc[2*i, 'oxytocin'] = df1_l.loc[i, 'oxytocin']
    df_m.loc[2*i+1, 'oxytocin'] = df1_l.loc[i, 'oxytocin']
    df_m.loc[2*i, 'quinine'] = df1_l.loc[i, 'quinine']
    df_m.loc[2*i+1, 'quinine'] = df1_l.loc[i, 'quinine']
    
    df_m.loc[2*i, 'w_1'] = df1_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_1'] = df1_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_1'] = df1_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_1'] = df1_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_1'] = df1_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_1'] = df1_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_1'] = df1_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_1'] = df1_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_1'] = df1_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_1'] = df1_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_2'] = df2_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_2'] = df2_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_2'] = df2_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_2'] = df2_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_2'] = df2_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_2'] = df2_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_2'] = df2_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_2'] = df2_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_2'] = df2_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_2'] = df2_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_3'] = df3_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_3'] = df3_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_3'] = df3_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_3'] = df3_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_3'] = df3_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_3'] = df3_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_3'] = df3_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_3'] = df3_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_3'] = df3_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_3'] = df3_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_4'] = df4_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_4'] = df4_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_4'] = df4_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_4'] = df4_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_4'] = df4_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_4'] = df4_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_4'] = df4_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_4'] = df4_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_4'] = df4_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_4'] = df4_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_5'] = df5_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_5'] = df5_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_5'] = df5_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_5'] = df5_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_5'] = df5_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_5'] = df5_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_5'] = df5_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_5'] = df5_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_5'] = df5_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_5'] = df5_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_6'] = df6_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_6'] = df6_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_6'] = df6_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_6'] = df6_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_6'] = df6_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_6'] = df6_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_6'] = df6_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_6'] = df6_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_6'] = df6_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_6'] = df6_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_8'] = df8_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_8'] = df8_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_8'] = df8_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_8'] = df8_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_8'] = df8_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_8'] = df8_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_8'] = df8_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_8'] = df8_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_8'] = df8_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_8'] = df8_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_9'] = df9_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_9'] = df9_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_9'] = df9_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_9'] = df9_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_9'] = df9_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_9'] = df9_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_9'] = df9_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_9'] = df9_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_9'] = df9_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_9'] = df9_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_10'] = df10_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_10'] = df10_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_10'] = df10_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_10'] = df10_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_10'] = df10_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_10'] = df10_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_10'] = df10_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_10'] = df10_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_10'] = df10_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_10'] = df10_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_11'] = df11_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_11'] = df11_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_11'] = df11_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_11'] = df11_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_11'] = df11_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_11'] = df11_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_11'] = df11_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_11'] = df11_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_11'] = df11_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_11'] = df11_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_12'] = df12_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_12'] = df12_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_12'] = df12_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_12'] = df12_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_12'] = df12_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_12'] = df12_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_12'] = df12_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_12'] = df12_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_12'] = df12_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_12'] = df12_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_13'] = df13_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_13'] = df13_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_13'] = df13_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_13'] = df13_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_13'] = df13_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_13'] = df13_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_13'] = df13_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_13'] = df13_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_13'] = df13_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_13'] = df13_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_14'] = df14_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_14'] = df14_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_14'] = df14_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_14'] = df14_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_14'] = df14_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_14'] = df14_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_14'] = df14_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_14'] = df14_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_14'] = df14_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_14'] = df14_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_15'] = df15_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_15'] = df15_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_15'] = df15_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_15'] = df15_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_15'] = df15_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_15'] = df15_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_15'] = df15_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_15'] = df15_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_15'] = df15_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_15'] = df15_l.loc[i, 'locomotive']

    df_m.loc[2*i, 'w_17'] = df17_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_17'] = df17_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_17'] = df17_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_17'] = df17_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_17'] = df17_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_17'] = df17_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_17'] = df17_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_17'] = df17_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_17'] = df17_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_17'] = df17_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_18'] = df18_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_18'] = df18_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_18'] = df18_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_18'] = df18_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_18'] = df18_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_18'] = df18_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_18'] = df18_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_18'] = df18_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_18'] = df18_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_18'] = df18_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_19'] = df19_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_19'] = df19_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_19'] = df19_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_19'] = df19_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_19'] = df19_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_19'] = df19_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_19'] = df19_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_19'] = df19_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_19'] = df19_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_19'] = df19_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_20'] = df20_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_20'] = df20_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_20'] = df20_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_20'] = df20_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_20'] = df20_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_20'] = df20_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_20'] = df20_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_20'] = df20_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_20'] = df20_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_20'] = df20_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_21'] = df21_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_21'] = df21_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_21'] = df21_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_21'] = df21_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_21'] = df21_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_21'] = df21_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_21'] = df21_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_21'] = df21_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_21'] = df21_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_21'] = df21_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_22'] = df22_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_22'] = df22_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_22'] = df22_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_22'] = df22_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_22'] = df22_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_22'] = df22_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_22'] = df22_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_22'] = df22_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_22'] = df22_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_22'] = df22_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_23'] = df23_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_23'] = df23_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_23'] = df23_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_23'] = df23_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_23'] = df23_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_23'] = df23_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_23'] = df23_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_23'] = df23_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_23'] = df23_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_23'] = df23_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_24'] = df24_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_24'] = df24_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_24'] = df24_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_24'] = df24_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_24'] = df24_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_24'] = df24_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_24'] = df24_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_24'] = df24_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_24'] = df24_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_24'] = df24_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_25'] = df25_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_25'] = df25_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_25'] = df25_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_25'] = df25_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_25'] = df25_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_25'] = df25_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_25'] = df25_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_25'] = df25_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_25'] = df25_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_25'] = df25_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_26'] = df26_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_26'] = df26_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_26'] = df26_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_26'] = df26_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_26'] = df26_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_26'] = df26_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_26'] = df26_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_26'] = df26_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_26'] = df26_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_26'] = df26_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_27'] = df27_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_27'] = df27_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_27'] = df27_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_27'] = df27_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_27'] = df27_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_27'] = df27_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_27'] = df27_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_27'] = df27_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_27'] = df27_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_27'] = df27_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_28'] = df28_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_28'] = df28_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_28'] = df28_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_28'] = df28_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_28'] = df28_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_28'] = df28_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_28'] = df28_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_28'] = df28_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_28'] = df28_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_28'] = df28_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_29'] = df29_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_29'] = df29_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_29'] = df29_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_29'] = df29_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_29'] = df29_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_29'] = df29_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_29'] = df29_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_29'] = df29_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_29'] = df29_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_29'] = df29_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_30'] = df30_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_30'] = df30_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_30'] = df30_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_30'] = df30_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_30'] = df30_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_30'] = df30_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_30'] = df30_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_30'] = df30_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_30'] = df30_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_30'] = df30_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_31'] = df31_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_31'] = df31_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_31'] = df31_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_31'] = df31_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_31'] = df31_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_31'] = df31_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_31'] = df31_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_31'] = df31_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_31'] = df31_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_31'] = df31_l.loc[i, 'locomotive']
    
    df_m.loc[2*i, 'w_32'] = df32_d.loc[i, 'water']
    df_m.loc[2*i+1, 'w_32'] = df32_l.loc[i, 'water']
    df_m.loc[2*i, 'a5_32'] = df32_d.loc[i, 'alcohol_5']
    df_m.loc[2*i+1, 'a5_32'] = df32_l.loc[i, 'alcohol_5']
    df_m.loc[2*i, 'a10_32'] = df32_d.loc[i, 'alcohol_10']
    df_m.loc[2*i+1, 'a10_32'] = df32_l.loc[i, 'alcohol_10']
    df_m.loc[2*i, 'a20_32'] = df32_d.loc[i, 'alcohol_20']
    df_m.loc[2*i+1, 'a20_32'] = df32_l.loc[i, 'alcohol_20']
    df_m.loc[2*i, 'loco_32'] = df32_d.loc[i, 'locomotive']
    df_m.loc[2*i+1, 'loco_32'] = df32_l.loc[i, 'locomotive']

In [429]:
df_m.to_excel(r'{}\drinkometer_df_25.xlsx'.format(path_), index=False, header=True)

In [430]:
path_ = 'C:\\Users\\mehrd\\PycharmProjects\\drinkometer\\data\\2021\\dataframes\\25'
df_25 = pd.read_excel("{}\drinkometer_df_25.xlsx".format(path_))

In [431]:
path_ = 'C:\\Users\\mehrd\\PycharmProjects\\drinkometer\\data\\2021\\dataframes\\26'
df_26 = pd.read_excel("{}\drinkometer_df_26.xlsx".format(path_))

In [432]:
df_a_25 = pd.DataFrame(index=range(7*30), columns=['day_index', 'strain', 'gender'
                                                'oxytocin', 'quinine', 'alcohol'])

df_a_26 = pd.DataFrame(index=range(7*30), columns=['day_index', 'strain', 'gender'
                                                'oxytocin', 'quinine', 'alcohol'])


In [433]:
df_a_state = pd.DataFrame(index=range(7*30*2), columns=['day_index', 'strain', 'gender',
                                                'oxytocin', 'quinine',
                                                'condition', 'alcohol'])

In [439]:
days = ([1]*30 + [2]*30 + [3]*30 + [4]*30 + [5]*30 + [6]*30 + [7]*30)*2
gender = ((['female']*14 + ['male']*16)*7)*2
strain = [str('wistar')] * 7 * 30 * 2
oxy = ['before'] * 210 + ['after'] * 210
condition = (['inject'] + ['inject'] + ['nasal'] + ['inject'] + ['nasal'] +
            ['control'] + ['nasal'] + ['inject'] + ['control'] + ['nasal'] + 
            ['nasal'] + ['inject'] + ['control'] + ['control'] + ['control'] +
            ['nasal'] + ['nasal'] + ['nasal'] + ['control'] + ['inject'] +
            ['inject'] + ['inject'] + ['inject'] + ['inject'] + ['nasal'] +
            ['control'] + ['inject'] + ['control'] + ['nasal'] + ['control']) * 14

In [338]:
df_a_25['day_index'] = days
df_a_25['gender'] = gender
df_a_25['strain'] = strain

In [346]:
df_a_26['day_index'] = days
df_a_26['gender'] = gender
df_a_26['strain'] = strain

In [440]:
df_a_state['day_index'] = days
df_a_state['gender'] = gender
df_a_state['strain'] = strain
df_a_state['oxytocin'] = oxy
df_a_state['condition'] = condition

In [339]:
counter = 0
for i in range(7):
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_1']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_1']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_1'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_2']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_2']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_2']) 
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_3']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_3']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_3'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_4']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_4']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_4'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_5']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_5']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_5'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_6']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_6']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_6'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_8']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_8']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_8'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_9']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_9']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_9'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_10']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_10']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_10'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_11']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_11']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_11'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_12']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_12']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_12'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_13']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_13']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_13'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_14']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_14']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_14'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_15']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_15']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_15'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_17']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_17']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_17'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_18']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_18']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_18'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_19']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_19']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_19'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_20']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_20']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_20'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_21']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_21']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_21'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_22']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_22']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_22'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_23']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_23']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_23'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_24']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_24']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_24'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_25']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_25']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_26']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_26']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_26'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_27']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_27']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_27'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_28']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_28']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_28'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_29']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_29']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_29'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_30']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_30']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_30'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_31']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_31']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_31'])
    counter += 1
    df_a_25.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_32']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_32']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_32'])
    counter += 1
        

In [347]:
counter = 0
for i in range(7):
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_1']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_1']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_1'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_2']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_2']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_2']) 
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_3']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_3']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_3'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_4']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_4']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_4'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_5']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_5']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_5'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_6']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_6']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_6'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_8']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_8']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_8'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_9']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_9']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_9'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_10']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_10']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_10'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_11']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_11']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_11'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_12']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_12']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_12'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_13']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_13']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_13'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_14']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_14']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_14'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_15']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_15']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_15'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_17']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_17']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_17'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_18']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_18']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_18'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_19']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_19']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_19'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_20']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_20']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_20'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_21']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_21']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_21'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_22']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_22']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_22'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_23']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_23']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_23'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_24']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_24']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_24'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_25']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_25']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_26']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_26']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_26'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_27']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_27']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_27'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_28']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_28']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_28'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_29']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_29']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_29'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_30']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_30']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_30'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_31']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_31']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_31'])
    counter += 1
    df_a_26.loc[counter, 'alcohol'] = np.nansum(df_m.loc[df_m['day_index'] == i+1]['a5_32']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a10_32']) + np.nansum(df_m.loc[df_m['day_index'] == i+1]['a20_32'])
    counter += 1
        

In [348]:
df_a_25.to_excel(r'{}\drinkometer_df_25_a_p.xlsx'.format(path_), index=False, header=True)
df_a_26.to_excel(r'{}\drinkometer_df_26_a_p.xlsx'.format(path_), index=False, header=True)

In [441]:
counter = 0
for i in range(7):
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_1']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_1']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_1'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_2']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_2']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_2']) 
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_3']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_3']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_3'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_4']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_4']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_4'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_5']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_5']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_5'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_6']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_6']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_6'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_8']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_8']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_8'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_9']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_9']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_9'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_10']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_10']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_10'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_11']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_11']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_11'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_12']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_12']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_12'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_13']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_13']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_13'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_14']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_14']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_14'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_15']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_15']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_15'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_17']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_17']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_17'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_18']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_18']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_18'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_19']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_19']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_19'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_20']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_20']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_20'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_21']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_21']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_21'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_22']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_22']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_22'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_23']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_23']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_23'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_24']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_24']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_24'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_25']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_25']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_25']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_25']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_27']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_27']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_27'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_28']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_28']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_28'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_29']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_29']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_29'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_30']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_30']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_30'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_31']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_31']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_31'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_25.loc[df_25['day_index'] == i+1]['a5_32']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a10_32']) + np.nansum(df_25.loc[df_25['day_index'] == i+1]['a20_32'])
    counter += 1
    

for i in range(7):
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_1']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_1']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_1'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_2']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_2']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_2']) 
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_3']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_3']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_3'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_4']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_4']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_4'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_5']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_5']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_5'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_6']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_6']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_6'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_8']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_8']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_8'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_9']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_9']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_9'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_10']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_10']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_10'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_11']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_11']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_11'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_12']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_12']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_12'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_13']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_13']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_13'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_14']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_14']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_14'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_15']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_15']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_15'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_17']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_17']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_17'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_18']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_18']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_18'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_19']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_19']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_19'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_20']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_20']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_20'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_21']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_21']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_21'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_22']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_22']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_22'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_23']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_23']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_23'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_24']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_24']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_24'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_25']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_25']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_25']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_25']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_25'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_27']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_27']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_27'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_28']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_28']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_28'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_29']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_29']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_29'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_30']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_30']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_30'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_31']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_31']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_31'])
    counter += 1
    df_a_state.loc[counter, 'alcohol'] = np.nansum(df_26.loc[df_26['day_index'] == i+1]['a5_32']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a10_32']) + np.nansum(df_26.loc[df_26['day_index'] == i+1]['a20_32'])
    counter += 1
        
        

In [437]:
df_a_state.to_excel(r'{}\drinkometer_df_state.xlsx'.format(path_), index=False, header=True)

In [243]:
df_a_25 = pd.DataFrame(index=range(7), columns=['date', 'day_index', 'strain', 'oxytocin', 'quinine',
                                                'a_1', 'a_2', 'a_3', 'a_4', 'a_5', 'a_6', 'a_8', 'a_9',
                                                'a_10', 'a_11', 'a_12', 'a_13', 'a_14', 'a_15', 'a_17',
                                                'a_18', 'a_19', 'a_20', 'a_21', 'a_22', 'a_23', 'a_24',
                                                'a_25', 'a_26', 'a_27', 'a_28', 'a_29', 'a_30', 'a_31',
                                                'a_32'])

df_a_26 = pd.DataFrame(index=range(7), columns=['date', 'day_index', 'strain', 'oxytocin', 'quinine',
                                                'a_1', 'a_2', 'a_3', 'a_4', 'a_5', 'a_6', 'a_8', 'a_9',
                                                'a_10', 'a_11', 'a_12', 'a_13', 'a_14', 'a_15', 'a_17',
                                                'a_18', 'a_19', 'a_20', 'a_21', 'a_22', 'a_23', 'a_24',
                                                'a_25', 'a_26', 'a_27', 'a_28', 'a_29', 'a_30', 'a_31',
                                                'a_32'])


In [244]:
for i in range(7):
    df_a_25.loc[i, 'date'] = df_m.loc[2*i, 'date']
    df_a_25.loc[i, 'day_index'] = df_m.loc[2*i, 'day_index']
    df_a_25.loc[i, 'strain'] = df_m.loc[2*i, 'strain']
    df_a_25.loc[i, 'oxytocin'] = df_m.loc[2*i, 'oxytocin']
    df_a_25.loc[i, 'quinine'] = df_m.loc[2*i, 'quinine']
    df_a_25.loc[i, 'gender'] = df_m.loc[2*i, 'quinine']
    
    
    df_a_25.loc[i, 'a_1'] = np.nansum(df_m.loc[2*i, 'a5_1': 'a20_1'])+ np.nansum(df_m.loc[2*i+1, 'a5_1': 'a20_1'])
    df_a_25.loc[i, 'a_2'] = np.nansum(df_m.loc[2*i, 'a5_2': 'a20_2'])+ np.nansum(df_m.loc[2*i+1, 'a5_1': 'a20_2'])
    df_a_25.loc[i, 'a_3'] = np.nansum(df_m.loc[2*i, 'a5_3': 'a20_3'])+ np.nansum(df_m.loc[2*i+1, 'a5_1': 'a20_3'])
    df_a_25.loc[i, 'a_4'] = np.nansum(df_m.loc[2*i, 'a5_3': 'a20_4'])+ np.nansum(df_m.loc[2*i+1, 'a5_1': 'a20_4'])
    
    df_a_25.loc[i, 'a_5'] = np.nansum(df_m.loc[2*i, 'a5_5': 'a20_5'])+ np.nansum(df_m.loc[2*i+1, 'a5_5': 'a20_5'])
    df_a_25.loc[i, 'a_6'] = np.nansum(df_m.loc[2*i, 'a5_6': 'a20_6'])+ np.nansum(df_m.loc[2*i+1, 'a5_6': 'a20_6'])
    df_a_25.loc[i, 'a_8'] = np.nansum(df_m.loc[2*i, 'a5_8': 'a20_8'])+ np.nansum(df_m.loc[2*i+1, 'a5_8': 'a20_8'])
    df_a_25.loc[i, 'a_9'] = np.nansum(df_m.loc[2*i, 'a5_9': 'a20_9'])+ np.nansum(df_m.loc[2*i+1, 'a5_9': 'a20_9'])
    
    df_a_25.loc[i, 'a_10'] = np.nansum(df_m.loc[2*i, 'a5_10': 'a20_10'])+ np.nansum(df_m.loc[2*i+1, 'a5_10': 'a20_10'])
    df_a_25.loc[i, 'a_11'] = np.nansum(df_m.loc[2*i, 'a5_11': 'a20_11'])+ np.nansum(df_m.loc[2*i+1, 'a5_11': 'a20_11'])
    df_a_25.loc[i, 'a_12'] = np.nansum(df_m.loc[2*i, 'a5_12': 'a20_12'])+ np.nansum(df_m.loc[2*i+1, 'a5_12': 'a20_12'])
    df_a_25.loc[i, 'a_13'] = np.nansum(df_m.loc[2*i, 'a5_13': 'a20_13'])+ np.nansum(df_m.loc[2*i+1, 'a5_13': 'a20_13'])
    
    df_a_25.loc[i, 'a_14'] = np.nansum(df_m.loc[2*i, 'a5_14': 'a20_14'])+ np.nansum(df_m.loc[2*i+1, 'a5_14': 'a20_14'])
    #df_a_25.loc[i, 'a_15'] = np.nansum(df_m.loc[2*i, 'a5_15': 'a20_15'])+ np.nansum(df_m.loc[2*i+1, 'a5_15': 'a20_15'])
    df_a_25.loc[i, 'a_17'] = np.nansum(df_m.loc[2*i, 'a5_17': 'a20_17'])+ np.nansum(df_m.loc[2*i+1, 'a5_17': 'a20_17'])
    df_a_25.loc[i, 'a_18'] = np.nansum(df_m.loc[2*i, 'a5_18': 'a20_18'])+ np.nansum(df_m.loc[2*i+1, 'a5_18': 'a20_18'])
    
    df_a_25.loc[i, 'a_19'] = np.nansum(df_m.loc[2*i, 'a5_19': 'a20_19'])+ np.nansum(df_m.loc[2*i+1, 'a5_19': 'a20_19'])
    df_a_25.loc[i, 'a_20'] = np.nansum(df_m.loc[2*i, 'a5_20': 'a20_20'])+ np.nansum(df_m.loc[2*i+1, 'a5_20': 'a20_20'])
    df_a_25.loc[i, 'a_21'] = np.nansum(df_m.loc[2*i, 'a5_21': 'a20_21'])+ np.nansum(df_m.loc[2*i+1, 'a5_21': 'a20_21'])
    df_a_25.loc[i, 'a_22'] = np.nansum(df_m.loc[2*i, 'a5_22': 'a20_22'])+ np.nansum(df_m.loc[2*i+1, 'a5_22': 'a20_22'])
    
    df_a_25.loc[i, 'a_23'] = np.nansum(df_m.loc[2*i, 'a5_23': 'a20_23'])+ np.nansum(df_m.loc[2*i+1, 'a5_23': 'a20_23'])
    df_a_25.loc[i, 'a_24'] = np.nansum(df_m.loc[2*i, 'a5_24': 'a20_24'])+ np.nansum(df_m.loc[2*i+1, 'a5_24': 'a20_24'])
    df_a_25.loc[i, 'a_25'] = np.nansum(df_m.loc[2*i, 'a5_25': 'a20_25'])+ np.nansum(df_m.loc[2*i+1, 'a5_25': 'a20_25'])
    df_a_25.loc[i, 'a_26'] = np.nansum(df_m.loc[2*i, 'a5_26': 'a20_26'])+ np.nansum(df_m.loc[2*i+1, 'a5_26': 'a20_26'])
    
    df_a_25.loc[i, 'a_27'] = np.nansum(df_m.loc[2*i, 'a5_27': 'a20_27'])+ np.nansum(df_m.loc[2*i+1, 'a5_27': 'a20_27'])
    df_a_25.loc[i, 'a_28'] = np.nansum(df_m.loc[2*i, 'a5_28': 'a20_28'])+ np.nansum(df_m.loc[2*i+1, 'a5_28': 'a20_28'])
    df_a_25.loc[i, 'a_29'] = np.nansum(df_m.loc[2*i, 'a5_29': 'a20_29'])+ np.nansum(df_m.loc[2*i+1, 'a5_29': 'a20_29'])
    df_a_25.loc[i, 'a_30'] = np.nansum(df_m.loc[2*i, 'a5_30': 'a20_30'])+ np.nansum(df_m.loc[2*i+1, 'a5_30': 'a20_30'])
    
    df_a_25.loc[i, 'a_31'] = np.nansum(df_m.loc[2*i, 'a5_31': 'a20_31'])+ np.nansum(df_m.loc[2*i+1, 'a5_31': 'a20_31'])
    df_a_25.loc[i, 'a_32'] = np.nansum(df_m.loc[2*i, 'a5_32': 'a20_32'])+ np.nansum(df_m.loc[2*i+1, 'a5_32': 'a20_32'])
    


In [246]:
for i in range(7):
    df_a_26.loc[i, 'date'] = df_m.loc[2*i, 'date']
    df_a_26.loc[i, 'day_index'] = df_m.loc[2*i, 'day_index']
    df_a_26.loc[i, 'strain'] = df_m.loc[2*i, 'strain']
    df_a_26.loc[i, 'oxytocin'] = df_m.loc[2*i, 'oxytocin']
    df_a_26.loc[i, 'quinine'] = df_m.loc[2*i, 'quinine']
    df_a_26.loc[i, 'gender'] = df_m.loc[2*i, 'quinine']
    
    
    df_a_26.loc[i, 'a_1'] = np.nansum(df_m.loc[2*i, 'a5_1': 'a20_1'])+ np.nansum(df_m.loc[2*i+1, 'a5_1': 'a20_1'])
    df_a_26.loc[i, 'a_2'] = np.nansum(df_m.loc[2*i, 'a5_2': 'a20_2'])+ np.nansum(df_m.loc[2*i+1, 'a5_1': 'a20_2'])
    df_a_26.loc[i, 'a_3'] = np.nansum(df_m.loc[2*i, 'a5_3': 'a20_3'])+ np.nansum(df_m.loc[2*i+1, 'a5_1': 'a20_3'])
    df_a_26.loc[i, 'a_4'] = np.nansum(df_m.loc[2*i, 'a5_3': 'a20_4'])+ np.nansum(df_m.loc[2*i+1, 'a5_1': 'a20_4'])
    
    df_a_26.loc[i, 'a_5'] = np.nansum(df_m.loc[2*i, 'a5_5': 'a20_5'])+ np.nansum(df_m.loc[2*i+1, 'a5_5': 'a20_5'])
    df_a_26.loc[i, 'a_6'] = np.nansum(df_m.loc[2*i, 'a5_6': 'a20_6'])+ np.nansum(df_m.loc[2*i+1, 'a5_6': 'a20_6'])
    df_a_26.loc[i, 'a_8'] = np.nansum(df_m.loc[2*i, 'a5_8': 'a20_8'])+ np.nansum(df_m.loc[2*i+1, 'a5_8': 'a20_8'])
    df_a_26.loc[i, 'a_9'] = np.nansum(df_m.loc[2*i, 'a5_9': 'a20_9'])+ np.nansum(df_m.loc[2*i+1, 'a5_9': 'a20_9'])
    
    df_a_26.loc[i, 'a_10'] = np.nansum(df_m.loc[2*i, 'a5_10': 'a20_10'])+ np.nansum(df_m.loc[2*i+1, 'a5_10': 'a20_10'])
    df_a_26.loc[i, 'a_11'] = np.nansum(df_m.loc[2*i, 'a5_11': 'a20_11'])+ np.nansum(df_m.loc[2*i+1, 'a5_11': 'a20_11'])
    df_a_26.loc[i, 'a_12'] = np.nansum(df_m.loc[2*i, 'a5_12': 'a20_12'])+ np.nansum(df_m.loc[2*i+1, 'a5_12': 'a20_12'])
    df_a_26.loc[i, 'a_13'] = np.nansum(df_m.loc[2*i, 'a5_13': 'a20_13'])+ np.nansum(df_m.loc[2*i+1, 'a5_13': 'a20_13'])
    
    df_a_26.loc[i, 'a_14'] = np.nansum(df_m.loc[2*i, 'a5_14': 'a20_14'])+ np.nansum(df_m.loc[2*i+1, 'a5_14': 'a20_14'])
    #df_a_26.loc[i, 'a_15'] = np.nansum(df_m.loc[2*i, 'a5_15': 'a20_15'])+ np.nansum(df_m.loc[2*i+1, 'a5_15': 'a20_15'])
    df_a_26.loc[i, 'a_17'] = np.nansum(df_m.loc[2*i, 'a5_17': 'a20_17'])+ np.nansum(df_m.loc[2*i+1, 'a5_17': 'a20_17'])
    df_a_26.loc[i, 'a_18'] = np.nansum(df_m.loc[2*i, 'a5_18': 'a20_18'])+ np.nansum(df_m.loc[2*i+1, 'a5_18': 'a20_18'])
    
    df_a_26.loc[i, 'a_19'] = np.nansum(df_m.loc[2*i, 'a5_19': 'a20_19'])+ np.nansum(df_m.loc[2*i+1, 'a5_19': 'a20_19'])
    df_a_26.loc[i, 'a_20'] = np.nansum(df_m.loc[2*i, 'a5_20': 'a20_20'])+ np.nansum(df_m.loc[2*i+1, 'a5_20': 'a20_20'])
    df_a_26.loc[i, 'a_21'] = np.nansum(df_m.loc[2*i, 'a5_21': 'a20_21'])+ np.nansum(df_m.loc[2*i+1, 'a5_21': 'a20_21'])
    df_a_26.loc[i, 'a_22'] = np.nansum(df_m.loc[2*i, 'a5_22': 'a20_22'])+ np.nansum(df_m.loc[2*i+1, 'a5_22': 'a20_22'])
    
    df_a_26.loc[i, 'a_23'] = np.nansum(df_m.loc[2*i, 'a5_23': 'a20_23'])+ np.nansum(df_m.loc[2*i+1, 'a5_23': 'a20_23'])
    df_a_26.loc[i, 'a_24'] = np.nansum(df_m.loc[2*i, 'a5_24': 'a20_24'])+ np.nansum(df_m.loc[2*i+1, 'a5_24': 'a20_24'])
    df_a_26.loc[i, 'a_25'] = np.nansum(df_m.loc[2*i, 'a5_25': 'a20_25'])+ np.nansum(df_m.loc[2*i+1, 'a5_25': 'a20_25'])
    df_a_26.loc[i, 'a_26'] = np.nansum(df_m.loc[2*i, 'a5_26': 'a20_26'])+ np.nansum(df_m.loc[2*i+1, 'a5_26': 'a20_26'])
    
    df_a_26.loc[i, 'a_27'] = np.nansum(df_m.loc[2*i, 'a5_27': 'a20_27'])+ np.nansum(df_m.loc[2*i+1, 'a5_27': 'a20_27'])
    df_a_26.loc[i, 'a_28'] = np.nansum(df_m.loc[2*i, 'a5_28': 'a20_28'])+ np.nansum(df_m.loc[2*i+1, 'a5_28': 'a20_28'])
    df_a_26.loc[i, 'a_29'] = np.nansum(df_m.loc[2*i, 'a5_29': 'a20_29'])+ np.nansum(df_m.loc[2*i+1, 'a5_29': 'a20_29'])
    df_a_26.loc[i, 'a_30'] = np.nansum(df_m.loc[2*i, 'a5_30': 'a20_30'])+ np.nansum(df_m.loc[2*i+1, 'a5_30': 'a20_30'])
    
    df_a_26.loc[i, 'a_31'] = np.nansum(df_m.loc[2*i, 'a5_31': 'a20_31'])+ np.nansum(df_m.loc[2*i+1, 'a5_31': 'a20_31'])
    df_a_26.loc[i, 'a_32'] = np.nansum(df_m.loc[2*i, 'a5_32': 'a20_32'])+ np.nansum(df_m.loc[2*i+1, 'a5_32': 'a20_32'])
    
    

In [250]:
df_a_25.to_excel(r'{}\drinkometer_df_25_a.xlsx'.format(path_), index=False, header=True)

In [252]:
df_a_26.to_excel(r'{}\drinkometer_df_26_a.xlsx'.format(path_), index=False, header=True)

In [ ]:
df.loc[(df['hour_index'] == hour)

In [449]:
df_a_state.replace(0, np.nan, inplace=True)

In [450]:
df_m = df_a_state.loc[(df_a_state['gender'] == str("male"))]
df_f = df_a_state.loc[(df_a_state['gender'] == str("female"))]

In [516]:
df_f.loc[242, 'alcohol'] = np.nan

C:\Users\mehrd\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation

x = "condition"
y = "alcohol"
hue = "oxytocin"

fig, ax = plt.subplots(figsize=(10,15))
ax = sns.boxplot(data=df_f, x=x, y=y, hue=hue)
add_stat_annotation(ax = ax, data=df_f, x=x, y=y, hue=hue,
                    box_pairs=[(("inject", "before"), ("inject", "after")),
                                 (("nasal", "before"), ("nasal", "after")),
                                 (("control", "before"), ("control", "after"))
                                ],
                    test='t-test_ind', text_format='full', loc='inside', verbose=2)
plt.savefig("female_alcoholdailyconsumption_orm.jpg", bbox_inches='tight' , dpi = 300)
plt.legend(loc='upper left', bbox_to_anchor=(1.03, 1))
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation

x = "condition"
y = "alcohol"
hue = "oxytocin"

fig, ax = plt.subplots(figsize=(10,15))
ax = sns.boxplot(data=df_m, x=x, y=y, hue=hue)
add_stat_annotation(ax = ax, data=df_m, x=x, y=y, hue=hue,
                    box_pairs=[(("inject", "before"), ("inject", "after")),
                                 (("nasal", "before"), ("nasal", "after")),
                                 (("control", "before"), ("control", "after"))
                                ],
                    test='t-test_ind', text_format='full', loc='inside', verbose=2)
plt.legend(loc='upper left', bbox_to_anchor=(1.03, 1))
plt.savefig("male_alcoholdailyconsumption.jpg", bbox_inches='tight' , dpi = 300)
plt.show()

In [506]:
df_m_inj_b = df_m.loc[(df_m['condition'] == str("inject")) & (df_m['oxytocin'] == str("before"))]
df_m_nasal_b = df_m.loc[(df_m['condition'] == str("nasal")) & (df_m['oxytocin'] == str("before"))]
df_m_cont_b = df_m.loc[(df_m['condition'] == str("control")) & (df_m['oxytocin'] == str("before"))]

df_m_inj_a = df_m.loc[(df_m['condition'] == str("inject")) & (df_m['oxytocin'] == str("after"))]
df_m_nasal_a = df_m.loc[(df_m['condition'] == str("nasal")) & (df_m['oxytocin'] == str("after"))]
df_m_cont_a = df_m.loc[(df_m['condition'] == str("control")) & (df_m['oxytocin'] == str("after"))]

df_f_inj_b = df_f.loc[(df_f['condition'] == str("inject")) & (df_f['oxytocin'] == str("before"))]
df_f_nasal_b = df_f.loc[(df_f['condition'] == str("nasal")) & (df_f['oxytocin'] == str("before"))]
df_f_cont_b = df_f.loc[(df_f['condition'] == str("control")) & (df_f['oxytocin'] == str("before"))]

df_f_inj_a = df_f.loc[(df_f['condition'] == str("inject")) & (df_f['oxytocin'] == str("after"))]
df_f_nasal_a = df_f.loc[(df_f['condition'] == str("nasal")) & (df_f['oxytocin'] == str("after"))]
df_f_cont_a = df_f.loc[(df_f['condition'] == str("control")) & (df_f['oxytocin'] == str("after"))]

In [ ]:
size = 20
sizee = 25
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
x = "day_index"
y = "alcohol"
#sns.set(font_scale = 1.4)
#sns.set_style("whitegrid", {'axes.grid' : False})
axes[0,0].get_shared_y_axes().join(axes[0,1])
axes[0,1].get_shared_y_axes().join(axes[0,2])



axes[0,0].grid(False)
#fig, ax = plt.subplots(figsize=(10,15))
sns.lineplot(data=df_m_cont_b, x=x, y=y, label = "male-control-baseline", ax=axes[0,0])
sns.lineplot(data=df_m_cont_a, x=x, y=y, label = "male-control-ADE", ax=axes[0,0])

plt.legend(loc='upper left')
axes[0,0].set_xlabel('')
axes[0,0].set_ylim([0, 4])
axes[0,0].set_ylabel("Ethyl-Alcohol (gr)",size=size)
axes[0,0].set_xticks([])



#axes[0,1].set_facecolor('grey')
axes[0,1].grid(False)
sns.lineplot(data=df_m_inj_b, x=x, y=y, label = "male-injection-baseline", ax=axes[0,1])
sns.lineplot(data=df_m_inj_a, x=x, y=y, label = "male-injection-ADE", ax=axes[0,1])
plt.legend(loc='upper left')

axes[0,1].set_ylim([0, 4])
axes[0,1].set_ylabel('')
axes[0,1].set_xlabel('')
axes[0,1].set_yticks([])
axes[0,1].set_xticks([])





axes[0,2].grid(False)
sns.lineplot(data=df_m_nasal_b, x=x, y=y, label = "male-nasal-baseline", ax=axes[0,2])
sns.lineplot(data=df_m_nasal_a, x=x, y=y, label = "male-nasal-ADE", ax=axes[0,2])
plt.legend(loc='upper left')

axes[0,2].set_ylim([0, 4])
axes[0,2].set_ylabel('')
axes[0,2].set_xlabel('')
axes[0,2].set_yticks([])
axes[0,2].set_xticks([])

axes[0,0].tick_params(axis ='both',which = 'both',labelsize=size)
axes[0,1].tick_params(axis ='both',which = 'both',labelsize=size)
axes[0,2].tick_params(axis ='both',which = 'both',labelsize=size)





axes[1,0].grid(False)
#fig, ax = plt.subplots(figsize=(10,15))
sns.lineplot(data=df_f_cont_b, x=x, y=y, label = "female-control-baseline", ax=axes[1,0])
sns.lineplot(data=df_f_cont_a, x=x, y=y, label = "female-control-ADE", ax=axes[1,0])

plt.legend(loc='upper left')

axes[1,0].set_ylim([0, 6])
axes[1,0].set_ylabel("Ethyl-Alcohol (gr)",size=size)
axes[1,0].set_xlabel("Days",size=size)



axes[1,1].grid(False)
#fig, ax = plt.subplots(figsize=(10,15))
sns.lineplot(data=df_f_inj_b, x=x, y=y, label = "female-injection-baseline", ax=axes[1,1])
sns.lineplot(data=df_f_inj_a, x=x, y=y, label = "female-injection-ADE", ax=axes[1,1])

plt.legend(loc='upper left')

axes[1,1].set_ylim([0, 6])
axes[1,1].set_yticks([])
axes[1,1].set_ylabel("")
axes[1,1].set_xlabel("Days",size=size)



df_f_nasal_a.loc[242, 'alcohol'] = np.nan
axes[1,2].grid(False)
#fig, ax = plt.subplots(figsize=(10,15))
sns.lineplot(data=df_f_nasal_b, x=x, y=y, label = "female-nasal-baseline", ax=axes[1,2])
sns.lineplot(data=df_f_nasal_a, x=x, y=y, label = "female-nasal-ADE", ax=axes[1,2])

plt.legend(loc='upper right')

axes[1,2].set_ylim([0, 6])
axes[1,2].set_yticks([])
axes[1,2].set_ylabel("")
axes[1,2].set_xlabel("Days",size=size)

axes[1,0].tick_params(axis ='both',which = 'both',labelsize=size)
axes[1,1].tick_params(axis ='both',which = 'both',labelsize=size)
axes[1,2].tick_params(axis ='both',which = 'both',labelsize=size)


plt.savefig("daily_comparison_alcoholdailyconsumption.jpg", bbox_inches='tight' , dpi = 600)

plt.show()


In [515]:
df_f_inj_a

,day_index,strain,gender,oxytocin,quinine,condition,alcohol
210,1,wistar,female,after,NaN,inject,NaN
211,1,wistar,female,after,NaN,inject,NaN
213,1,wistar,female,after,NaN,inject,NaN
217,1,wistar,female,after,NaN,inject,0.486443
221,1,wistar,female,after,NaN,inject,NaN
240,2,wistar,female,after,NaN,inject,2.601160
241,2,wistar,female,after,NaN,inject,1.989874
243,2,wistar,female,after,NaN,inject,2.102519
247,2,wistar,female,after,NaN,inject,2.175630
251,2,wistar,female,after,NaN,inject,4.569804
